In [2]:
import numpy as np

# <center> TP2 - Chiffrement par substitution polyalphabétique </center>
<center> 2023/2024 - L. Naert, T. Ferragut, T. Godin </center>

_Certains exemples et textes de ce TP sont tirés de Exercices et Problèmes de cryptographie de Damien Vergnaud, 3ème édition_

Dans le chiffrement par substitution monoalphabétique, un symbole du message en clair était toujours remplacé par un même symbole pour constituer le message chiffré et un symbole du chiffré correspondait toujours au même symbole du clair. L'attaque par analyse de fréquence est donc très efficace sur ce type de chiffrement. 

Dans le chiffrement par substitution polyalphabétique, plusieurs alphabets de chiffrement sont utilisés. Un symbole du message en clair sera remplacé par un autre symbole en fonction de sa position dans le clair, de l'algorithme de chiffrement utilisé et de la clef choisi. Ainsi, deux occurences d'un même symbole dans le clair peuvent être remplacées par des symboles différents dans le chiffré. Et un même symbole dans le chiffré peut servir à coder des lettres différentes du clair. 

Dans ce TP, nous étudierons 2 types de chiffrement par substitution polyalphabétique : le chiffrement de Vigenère et le chiffrement de Hill.

Pour chaque type de chiffrement, nous développerons des fonctions pour:
1. chiffrer un message en clair à l'aide d'une clef
2. déchiffrer un message (en connaissant la clef)
3. (décrypter un message (ne connaissant pas la clef)) $\to$ partie optionnelle et seulement pour Vigenère


Par convention, nous appelerons :
- $k$ : la clef
- $E_k$ : la fonction de chiffrement
- $D_k$ : la fonction de déchiffrement
- $m$ : le message en clair
- $m_i$ : la lettre de rang $i$ sur message en clair
- $c$ : le message chiffré
- $c_i$ : la lettre de rang $i$ sur message chiffré



## 1 - Chiffrement de Vigenère

Le chiffrement de Vigenère consiste à effectuer de façon périodique plusieurs chiffrements par décalage. La clef utilisée est un mot, bien plus court que le message à chiffrer, qui donne la valeur du décalage à effectuer pour chaque lettre du clair.

Par exemple, chiffrons le mot __chiffrement__ à l'aide de l'algorithme de Vigenère et de la clef __clef__. Cette clef indique que :
- la première lettre du clair sera chiffrée à l'aide d'un décalage de 2 positions ('c' est codé par "2" dans $\mathbb{Z}/26\mathbb{Z}$), 
- la deuxième lettre subira un décalage de 11 positions ('l' est codé par "11"), 
- la troisième de 4 positions ('e' est codé par "4"), 
- la quatrième de 5 positions (f est codé par "5"), 
- et on recommence : la cinquième de 2 positions,
- la sixième de 11 positions
- etc.

Le mot __chiffrement__ sera donc chiffré __esmkhcirgyx__. On peut remarquer que les deux "f" du clair sont chiffrés par "k" et "h" respectivement. 

In [3]:
# Quelques fonctions utiles (ou pas)
def lettreToEntier(lettre, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    return alphabet.find(lettre)
def entierToLettre(a, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    return alphabet[int(a)]

print("Lettre -> entier : w ->", lettreToEntier('w'))
print("Entier -> Lettre : 22 ->", entierToLettre(22))

Lettre -> entier : w -> 22
Entier -> Lettre : 22 -> w


### 1 - a) Chiffrement


> __Question 1 (chiffrement)__ : Définir une fonction `chiffrementVigenere(msgClair, clef, alphabet)` qui étant donné un message en clair `msgClair`, un mot `clef` et un alphabet `alphabet` (par défaut, l'alphabet français) renvoie le message chiffré par l'algorithme de Vigenère.

_Note_ : il est bien sûr possible et conseillé de réutiliser les fonctions codées dans le TP précédent.

In [4]:
def chiffrementVigenere(msg, clef, alphabet="abcdefghijklmnopqrstuvwxyz"):
    c = ''
    for i,char in enumerate(msg) : 
        index = alphabet.index(char)
        
        k = alphabet.index(clef[i%len(clef)])
        
        c += alphabet[(index+k)%len(alphabet)]
    return c


chiffrementVigenere("vigenere","truc")

'ozaggvlg'

In [5]:
try:
    assert chiffrementVigenere("vigenere","truc") == "ozaggvlg"
    assert chiffrementVigenere("chiffrement","clef") == "esmkhcirgyx"
    print("chiffrementVigenere : OK")
except:
    print("chiffrementVigenere : ERREUR")


chiffrementVigenere : OK


### 1 - b) Déchiffrement

Partant du message chiffré et connaissant la clef de chiffrement, il est possible de déchiffrer le message en opérant le décalage inverse.

> __Question 2 (déchiffrement)__ : Définir une fonction `dechiffrementVigenere(msgChiffre, clef, alphabet)` qui étant donné un message chiffré `msgChiffre`, la clef qui a servi à construire ce message chiffré `clef` et un alphabet `alphabet` (par défaut, l'alphabet français) renvoie le message en clair correspondant.

In [6]:
def dechiffrementVigenere(cypher, clef, alphabet="abcdefghijklmnopqrstuvwxyz"):
    m = ''
    for i,char in enumerate(cypher) : 
        index = alphabet.index(char)
        
        k = alphabet.index(clef[i%len(clef)])
        
        m += alphabet[(index-k)%len(alphabet)]
    return m

dechiffrementVigenere("esmkhcirgyx","clef")


'chiffrement'

In [7]:
try:
    assert dechiffrementVigenere("esmkhcirgyx","clef") == "chiffrement"
    assert dechiffrementVigenere("ozaggvlg","truc") == "vigenere"
    print("dechiffrementVigenere : OK")
except:
    print("dechiffrementVigenere : ERREUR")
    
  

dechiffrementVigenere : OK


### 1 - c) Attaque : taille de la clef et analyse de fréquence

__-----------  Début de la partie optionnelle  ---------------__


__Attention, cette partie n'est à faire que si le reste du TP (dont Chiffrement de Hill en partie 2) est terminé.__

Une attaque directe par analyse de fréquence sur l'ensemble du chiffré ne sera pas efficace sur un message chiffré avec l'algorithme de  Vigenère car une même lettre du clair peut-être chiffrée par des lettres différentes et une même lettre du chiffré peut représenter des lettres du clair distinctes.

Mais, si l'on connait le nombre de lettres $l$ que comporte la clef, il devient possible d'effectuer des analyses de fréquences efficaces sur chacun des sous-messages chiffrés déterminés en prenant les lettres du message clair espacées de $l$. Il y a donc $l$ sous-messages chiffrés à cryptanalyser.

L'attaque pour décrypter un message codé avec Vigenère consiste donc tout d'abord à déterminer la taille de la clef et à ensuite effectuer une attaque similaire à celle du chiffrement par décalage (analyse de fréquence) sur chaque sous message chiffré.

__Exemple__ : Si le chiffré est :"dbsndqufgdi" et que la clef est de longueur $3$. Il faudra faire 3 analyses de fréquences (il s'agit d'un exemple totalement fictif, le message est trop court pour pouvoir faire une analyse de fréquence, encore moins 3...) : 
- la première sur "dnud"
- la deuxième sur "bdfi"
- la troisière sur "sqg"



Le _test de Kasiski_ est une méthode pour déterminer la longueur de la clef. Il repose sur le fait que si plusieurs groupes de lettres sont égaux dans le chiffré, ils correspondent certainement au même groupe de lettre dans le clair chiffré avec la même partie de la clef. La taille de l'intervalle séparant les motifs identiques sera donc probablement un multiple de la taille de la clef. S'il y a plusieurs répétitions de motifs, le plus grand diviseur commun de ces longeurs d'intervalle est possiblement la taille de la clef.

> __Question 3 (Test de Kasiski)__ : Définir une fonction `testKasiski(msgChiffre)` qui étant donné un message chiffré `msgChiffre` renvoie la taille de clef la plus probable.

In [8]:
def testKasiski(msgChiffre):
    return "TODO"

c = "zbpuevpuqsdlzgllksousvpasfpddggaqwptdgptzweemqzrdjtddefekeferdprrcyndgluaowcnbptzzzrbvpssfpashpncotemhaeqrferdlrlwwertlussfikgoeuswotfdgqsyasrlnrzppdhtticfrciwurhcezrpmhtpuwiyenamrdbzyzwelzucamrptzqseqcfgdrfrhrpatsepzgfnaffisbpvdblisrplzgnemswaqoxpdseehbeeksdptdttqsdddgxurwnidbdddplncsd"

try:
    assert testKasiski(c) == 4
    print("testKasiski : OK")
except:
    print("testKasiski : ERREUR")



testKasiski : ERREUR


> __Question 4 (attaque par analyse de fréquence avec longueur de clef connue)__ : Définir une fonction `attaqueFrequenceVig(msgChiffre, l, frequenceLangue, alphabet)` qui décrypte le message en paramètre en effectuant des analyses de fréquences connaissant la longueur `l` de la clef ayant servie à chiffrer le message. `frequenceLangue` est un dictionnaire des fréquences dans la langue du message (le dictionnaire des fréquences de chaque lettre en français `frequenceFrancais` vous est fourni).

_Note : n'hésitez pas à utiliser des fonctions écrites dans le TP précédent._

In [9]:
frequenceFrancais = {'a': 8.15, 'b': 0.97, 'c': 3.15, 'd': 3.73, 'e': 17.39, 
                     'f': 1.12, 'g': 0.97, 'h': 0.85, 'i': 7.31, 'j': 0.45, 
                     'k': 0.02, 'l': 5.69, 'm': 2.87, 'n': 7.12, 'o': 5.28, 
                     'p': 2.80, 'q': 1.21, 'r': 6.64, 's': 8.14, 't': 7.22, 
                     'u': 6.38, 'v': 1.64, 'w': 0.03, 'x': 0.41, 'y': 0.28, 
                     'z': 0.15}


def attaqueFrequenceVig(msgChiffre, l, frequenceLangue = frequenceFrancais, alphabet="abcdefghijklmnopqrstuvwxyz"):
    return "TODO"

c = "zbpuevpuqsdlzgllksousvpasfpddggaqwptdgptzweemqzrdjtddefekeferdprrcyndgluaowcnbptzzzrbvpssfpashpncotemhaeqrferdlrlwwertlussfikgoeuswotfdgqsyasrlnrzppdhtticfrciwurhcezrpmhtpuwiyenamrdbzyzwelzucamrptzqseqcfgdrfrhrpatsepzgfnaffisbpvdblisrplzgnemswaqoxpdseehbeeksdptdttqsdddgxurwnidbdddplncsd"

try:
    assert attaqueFrequenceVig(c,4) == "aneufheureslasalledutheatredesvarietesetaitencorevidequelquespersonnesaubalconetalorchestreattendaientperduesparmilesfauteuilsdeveloursgrenatdanslepetitjourdulustreademifeuxuneombrenoyaitlagrandetacherougedurideauetpasunbruitnevenaitdelascenelarampeeteintelespupitresdesmusiciensdebandes"
    print("attaqueFrequenceVig : OK")
except:
    print("attaqueFrequenceVig : ERREUR")

attaqueFrequenceVig : ERREUR


__-----------  Fin de la partie optionnelle  -----------__

## 2 - Chiffrement de Hill

Le chiffrement de Hill est un type de chiffrement par substitution polyalphabétique où les lettres du clair sont chiffrées et déchiffrées par paquets et non les unes à la suite des autres. Ce chiffrement est donc mieux protégé contre les attaques par analyse de fréquence.

Pour chiffrer, on commence par choisir une matrice carrée inversible (nous verrons plus tard comment vérifier qu'une matrice est inversible dans $\mathbb{Z}/26\mathbb{Z}$) de taille $p \times p$. Cette matrice constitue la clef de chiffrement. Ensuite, le message en clair est divisé en blocs/vecteurs de longueur $p$. Le dernier bloc est éventuellement complété avec une lettre choisie arbitrairement si sa longueur est différente de $p$. Chaque vecteur est chiffré en le multipliant avec la matrice carré. Evidemment, pour effectuer la multiplication matricielle, les lettres des messages sont converties en nombre entre 0 et 25. 

La fonction de chiffrement correspondante pour un bloc de $p$ lettres est : 

\begin{align*}
  E \colon (\mathbb{Z}/26\mathbb{Z})^p &\to (\mathbb{Z}/26\mathbb{Z})^p\\
    \begin{pmatrix}
m_i\\
m_{i+1}\\
... \\
m_{i+p-1} \\
\end{pmatrix} & \mapsto \begin{pmatrix}
c_i\\
c_{i+1}\\
... \\
c_{i+p-1} \\
\end{pmatrix} = K \times     \begin{pmatrix}
m_i\\
m_{i+1}\\
... \\
m_{i+p-1} \\
\end{pmatrix}
\end{align*}

où $K$ est une matrice carrée inversible de taille $p \times p$.

Par exemple, Chiffrons le message "TEXTE" avec une matrice $K \in \mathcal{M_{2,2}}$ : 
$\begin{pmatrix}
3 & 5\\
6 & 17\\
\end{pmatrix}$.

Nous commençons par faire des blocs de 2 lettres : "TE","XT","EW" (ici, on rajoute un W en fin de message pour avoir un dernier bloc de deux lettres car le message initial n'a pas un nombre pair de lettres). 

"TE" correspond au couple de valeurs (19,4). 


$
\begin{pmatrix}
3 & 5\\
6 & 17\\
\end{pmatrix} \times
\begin{pmatrix}
19\\
4\\
\end{pmatrix} = 
\begin{pmatrix}
77\\
182\\
\end{pmatrix}
$


Il faut ensuite convertir le résultat $\begin{pmatrix}
77\\
182\\
\end{pmatrix}$ en nombre de $\mathbb{Z}/26\mathbb{Z}$ :$\begin{pmatrix}
25\\
0\\
\end{pmatrix}$ puis en lettre : $\begin{pmatrix}
Z\\
A\\
\end{pmatrix}.$
"TE" sera donc chiffré "ZA". 
En recommançant le processus pour les autres couples de lettres, le clair "TEXTE" devient le chiffré "ZAITSI".




### 2 - a) Chiffrement

> __Question 5 (chiffrement)__ : Définir une fonction `chiffrementHill(msgClair, clef, lettreRemplacement, alphabet)` qui étant donné un message en clair `msgClair`, une clef de chiffrement `clef` sous forme de matrice carré $p \times p$, une lettre de "padding" (par défaut "w") s'il n'est pas possible de diviser le message clair en blocs de taille p et un alphabet `alphabet` (par défaut, l'alphabet français), renvoie le message chiffré correspondant.

In [10]:
import numpy as np

def chiffrementHill(clair, clef, lettrePadding = 'w', alphabet = "abcdefghijklmnopqrstuvwxyz") :
    
    # Add padding
    while len(clair) % len(clef) != 0 : 
        clair += lettrePadding
    
    # Split every n chars
    segments = []
    for k in range(0,len(clair),len(clef)) :
        segments.append(clair[k:k+len(clef)])
        
    # Crypt
    cypher_int = []
    for segment in segments :
        lst = []
        for char in segment:
            lst.append(lettreToEntier(char))
        cypher_int.append((clef@np.array(lst))%len(alphabet))
        
    # Convert cypher int matrix to cypher text
    cypher = ""
    for cypher_segment in cypher_int : 
        for decimal in cypher_segment : 
            cypher += entierToLettre(decimal)
    return cypher

chiffrementHill("texte",np.array([[3,5],[6,17]]))


'zaitsi'

In [11]:
try:
    assert chiffrementHill("texte",np.array([[3,5],[6,17]])) == "zaitsi"
    assert chiffrementHill("chiffredehill",np.array([[1,3,1],[1,1,0],[2,9,3]])) == "fjnlkcrhvqppvha"
    print("chiffrementHill : OK")
except:
    print("chiffrementHill : ERREUR")


chiffrementHill : OK


### 2 - b) Déchiffrement


Partant du message chiffré $c$ et connaissant la matrice de chiffrement de taille $p \times p $, il est possible de déchiffrer le message pour obtenir le clair $m$.

Il s'agit de diviser le message chiffré en vecteurs de $p$ lettres et de multiplier ces vecteurs par l'__inverse__ de la matrice de chiffrement.

La fonction de déchiffrement d'un bloc de $p$ lettres est ainsi : 

\begin{align*}
  D \colon (\mathbb{Z}/26\mathbb{Z})^p &\to (\mathbb{Z}/26\mathbb{Z})^p\\
    \begin{pmatrix}
c_i\\
c_{i+1}\\
... \\
c_{i+p-1} \\
\end{pmatrix} & \mapsto \begin{pmatrix}
m_i\\
m_{i+1}\\
... \\
m_{i+p-1} \\
\end{pmatrix} = K^{-1} \times     \begin{pmatrix}
c_i\\
c_{i+1}\\
... \\
c_{i+p-1} \\
\end{pmatrix}
\end{align*}

où $ K^{-1}$ est l'inverse de la matrice de chiffrement K dans $\mathbb{Z}/26\mathbb{Z}$.


Pour déchiffrer le message, nous avons donc besoin d'inverser la matrice K dans $\mathbb{Z}/26\mathbb{Z}$. 


__La matrice K est inversible dans $\mathbb{Z}/n\mathbb{Z}$ si son déterminant est inversible dans $\mathbb{Z}/n\mathbb{Z}$.__


> __Question 6 :__ Faire une fonction `estInversibleMat(k, n)` qui renvoie `True` si la matrice carrée $k$ est inversible sur $\mathbb{Z}/n\mathbb{Z}$ et `False`sinon.

In [12]:
def det(mat,n) : 
    try :
        mat = mat.tolist()
    except : 
        pass
    if len(mat) == 2 : 
        return (mat[0][0]*mat[1][1]-mat[0][1]*mat[1][0]) % n
    somme = 0
    for i in range(len(mat)):
        tmp_mat = [line[1:] for line in mat]
        tmp_mat.pop(i)
        if i % 2 == 0 :
            somme += mat[i][0]*det(tmp_mat,n)
        else : 
            somme -= mat[i][0]*det(tmp_mat,n)
    return somme % n


def estInversibleMat(k,n):
    """
    k est une matrice carree
    """
    a = det(k,n)
    while n != 0 : 
        a, n = n, a % n
    return a == 1

estInversibleMat(np.array([[1,3,1],[1,1,0],[2,9,3]]),26)

True

In [13]:
try:
    assert estInversibleMat(np.array([[3,5],[6,17]]),26) == True
    assert estInversibleMat(np.array([[1,3,1],[1,1,0],[2,9,3]]),26) == True
    assert estInversibleMat(np.array([[1,2],[3,4]]),26) == False
    print("estInversibleMat : OK")
except:
    print("estInversibleMat : ERREUR")


estInversibleMat : OK


__Pour inverser K, il faut multiplier la transposée de la comatrice de K par un inverse modulaire de son déterminant.__


> __Question 7 :__ A partir de la fonction `comatrice` fournie, faire une fonction `inverseMat(k, n)` qui teste si k est inversible. Si oui, renvoie la matrice inverse de k dans $\mathbb{Z}/n\mathbb{Z}$ et sinon, renvoie -1. N'hésitez pas à réutiliser des fonctions du TP précédent.

In [98]:
def comatrice(mat,n):
    det = np.linalg.det(mat)
    if(det!=0):
        return (np.linalg.inv(mat).T * det)%n
    else:
        return "Matrice non inversible"
        
#print("Matrice des cofacteurs/Comatrice :\n", comatrice([[1, 2], [3, 4]]))

def inverse_modulo(a, n):
    # Initialisation des variables
    r, r_prev = n, a
    x, x_prev = 0, 1

    while r != 0:
        quotient = r_prev // r
        r, r_prev = r_prev - quotient * r, r
        x, x_prev = x_prev - quotient * x, x

    if r_prev > 1:
        raise ValueError("L'inverse modulo n'existe pas.")
    if x_prev < 0:
        x_prev += n

    return x_prev


def transposeMat(matrix) : 
    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]


def inverseMat(k,n):
    if not estInversibleMat(k,n) : 
        return -1 
    determinant = det(k,n)
    # print(determinant)
    inverse_det = inverse_modulo(determinant,n)
    # print(inverse_det)
    comat = comatrice(k,n)
    # print(comat)
    transpose = np.array(transposeMat(comat))
    # print(transpose)
    return ((inverse_det * transpose)%n)


print((inverseMat([[9,4],[5,7]],26)@np.array([[9,4],[5,7]]))%26)
print((inverseMat([[3,7],[5,8]],26)@np.array([[3,7],[5,8]]))%26)
print((inverseMat(np.array([[1,3,1],[1,1,0],[2,9,3]]),26)@np.array([[1,3,1],[1,1,0],[2,9,3]]))%26)

[[1.00000000e+00 1.13686838e-13]
 [4.54747351e-13 1.00000000e+00]]
[[1.00000000e+00 0.00000000e+00]
 [2.84217094e-14 1.00000000e+00]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [99]:
try:
    assert (inverseMat([[9,4],[5,7]],26) == np.array([[ 5, 12],[15, 25]])).all
    assert (inverseMat([[3,7],[5,8]],26) == np.array([[ 4, 3],[17, 21]])).all
    assert (inverseMat(np.array([[1,3,1],[1,1,0],[2,9,3]]),26) == np.array([[ 3,0,25],[23,1,1],[ 7,23,24]])).all
    print("inverseMat : OK")
except:
    print("inverseMat : ERREUR")
    
    

inverseMat : OK


> __Question 8 (déchiffrement)__ : Définir une fonction `dechiffrementHill(msgChiffre, clef, alphabet)` qui étant donné un message chiffré `msgChiffre`, la matrice de chiffrement `clef` qui a servi à construire ce message chiffré et un alphabet `alphabet` (par défaut, l'alphabet français) renvoie le message en clair correspondant.

In [115]:
def dechiffrementHill(cypher, clef, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    inverse_clef = np.round(inverseMat(clef,len(alphabet)))
    print(inverse_clef)
    return chiffrementHill(cypher,inverse_clef,alphabet=alphabet)

print(dechiffrementHill("zaitsi",np.array([[3,5],[6,17]])))
print(dechiffrementHill("fjnlkcrhvqppvha",np.array([[1,3,1],[1,1,0],[2,9,3]])))

[[ 7.  1.]
 [22. 15.]]
textew
[[ 3.  0. 25.]
 [23.  1.  1.]
 [ 7. 23. 24.]]
chiffredehillww


In [114]:
try:
    assert dechiffrementHill("zaitsi",np.array([[3,5],[6,17]])) == "textew"
    assert dechiffrementHill("fjnlkcrhvqppvha",np.array([[1,3,1],[1,1,0],[2,9,3]])) == "chiffredehillww"
    print("dechiffrementHill : OK")
except:
    print("dechiffrementHill : ERREUR")

dechiffrementHill : OK
